In [44]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from collections import namedtuple
import math as mt
import joblib

In [15]:
def compute_face_coords(mesh):
    """ Compute coordinates of elements (triangles)
    Parameters:
    -----------
    mesh: mesh object
        fesom mesh object
    Returns:
    --------
    face_x: numpy array
        x coordinates
    face_y: numpy array
        y coordinates
    """
    first_mean = mesh.x2[mesh.elem].mean(axis=1)
    j = np.where(np.abs(mesh.x2[mesh.elem][:, 0] - first_mean) > 100)[0]
    cyclic_elems = mesh.x2[mesh.elem][j].copy()
    new_means = np.where(cyclic_elems > 0, cyclic_elems, cyclic_elems + 360).mean(
        axis=1
    )
    new_means[new_means > 180] = new_means[new_means > 180] - 360
    first_mean[j] = new_means
    face_x = first_mean
    face_y = mesh.y2[mesh.elem].mean(axis=1)
    return face_x, face_y

In [33]:
def scalar_r2g(al, be, ga, rlon, rlat):
    """
    Converts rotated coordinates to geographical coordinates.
    Parameters
    ----------
    al : float
        alpha Euler angle
    be : float
        beta Euler angle
    ga : float
        gamma Euler angle
    rlon : array
        1d array of longitudes in rotated coordinates
    rlat : array
        1d araay of latitudes in rotated coordinates
    Returns
    -------
    lon : array
        1d array of longitudes in geographical coordinates
    lat : array
        1d array of latitudes in geographical coordinates
    """

    rad = mt.pi / 180
    al = al * rad
    be = be * rad
    ga = ga * rad
    rotate_matrix = np.zeros(shape=(3, 3))
    rotate_matrix[0, 0] = np.cos(ga) * np.cos(al) - np.sin(ga) * np.cos(be) * np.sin(al)
    rotate_matrix[0, 1] = np.cos(ga) * np.sin(al) + np.sin(ga) * np.cos(be) * np.cos(al)
    rotate_matrix[0, 2] = np.sin(ga) * np.sin(be)
    rotate_matrix[1, 0] = -np.sin(ga) * np.cos(al) - np.cos(ga) * np.cos(be) * np.sin(
        al
    )
    rotate_matrix[1, 1] = -np.sin(ga) * np.sin(al) + np.cos(ga) * np.cos(be) * np.cos(
        al
    )
    rotate_matrix[1, 2] = np.cos(ga) * np.sin(be)
    rotate_matrix[2, 0] = np.sin(be) * np.sin(al)
    rotate_matrix[2, 1] = -np.sin(be) * np.cos(al)
    rotate_matrix[2, 2] = np.cos(be)

    rotate_matrix = np.linalg.pinv(rotate_matrix)

    rlat = rlat * rad
    rlon = rlon * rad

    # Rotated Cartesian coordinates:
    xr = np.cos(rlat) * np.cos(rlon)
    yr = np.cos(rlat) * np.sin(rlon)
    zr = np.sin(rlat)

    # Geographical Cartesian coordinates:
    xg = rotate_matrix[0, 0] * xr + rotate_matrix[0, 1] * yr + rotate_matrix[0, 2] * zr
    yg = rotate_matrix[1, 0] * xr + rotate_matrix[1, 1] * yr + rotate_matrix[1, 2] * zr
    zg = (
        rotate_matrix[2, 0] * xr + rotate_matrix[2, 1] * yr + rotate_matrix[2, 2] * zr
    )

    # Geographical coordinates:
    lat = np.arcsin(zg)
    lon = np.arctan2(yg, xg)

    a = np.where((np.abs(xg) + np.abs(yg)) == 0)
    if a:
        lon[a] = 0

    lat = lat / rad
    lon = lon / rad

    return (lon, lat)

In [31]:
def vec_rotate_r2g(al, be, ga, lon, lat, urot, vrot, flag):
    """
    Rotate vectors from rotated coordinates to geographical coordinates.
    Parameters
    ----------
    al : float
        alpha Euler angle
    be : float
        beta Euler angle
    ga : float
        gamma Euler angle
    lon : array
        1d array of longitudes in rotated or geographical coordinates (see flag parameter)
    lat : array
        1d array of latitudes in rotated or geographical coordinates (see flag parameter)
    urot : array
        1d array of u component of the vector in rotated coordinates
    vrot : array
        1d array of v component of the vector in rotated coordinates
    flag : 1 or 0
        flag=1  - lon,lat are in geographical coordinate
        flag=0  - lon,lat are in rotated coordinate
    Returns
    -------
    u : array
        1d array of u component of the vector in geographical coordinates
    v : array
        1d array of v component of the vector in geographical coordinates
    """

    #   first get another coordinate
    if flag == 1:
        (rlon, rlat) = scalar_g2r(al, be, ga, lon, lat)
    else:
        rlon = lon
        rlat = lat
        (lon, lat) = scalar_r2g(al, be, ga, rlon, rlat)

    #   then proceed...
    rad = mt.pi / 180
    al = al * rad
    be = be * rad
    ga = ga * rad

    rotate_matrix = np.zeros(shape=(3, 3))
    rotate_matrix[0, 0] = np.cos(ga) * np.cos(al) - np.sin(ga) * np.cos(be) * np.sin(al)
    rotate_matrix[0, 1] = np.cos(ga) * np.sin(al) + np.sin(ga) * np.cos(be) * np.cos(al)
    rotate_matrix[0, 2] = np.sin(ga) * np.sin(be)
    rotate_matrix[1, 0] = -np.sin(ga) * np.cos(al) - np.cos(ga) * np.cos(be) * np.sin(
        al
    )
    rotate_matrix[1, 1] = -np.sin(ga) * np.sin(al) + np.cos(ga) * np.cos(be) * np.cos(
        al
    )
    rotate_matrix[1, 2] = np.cos(ga) * np.sin(be)
    rotate_matrix[2, 0] = np.sin(be) * np.sin(al)
    rotate_matrix[2, 1] = -np.sin(be) * np.cos(al)
    rotate_matrix[2, 2] = np.cos(be)

    rotate_matrix = np.linalg.pinv(rotate_matrix)
    rlat = rlat * rad
    rlon = rlon * rad
    lat = lat * rad
    lon = lon * rad

    #   vector in rotated Cartesian
    txg = -vrot * np.sin(rlat) * np.cos(rlon) - urot * np.sin(rlon)
    tyg = -vrot * np.sin(rlat) * np.sin(rlon) + urot * np.cos(rlon)
    tzg = vrot * np.cos(rlat)

    #   vector in geo Cartesian
    txr = (
        rotate_matrix[0, 0] * txg
        + rotate_matrix[0, 1] * tyg
        + rotate_matrix[0, 2] * tzg
    )
    tyr = (
        rotate_matrix[1, 0] * txg
        + rotate_matrix[1, 1] * tyg
        + rotate_matrix[1, 2] * tzg
    )
    tzr = (
        rotate_matrix[2, 0] * txg
        + rotate_matrix[2, 1] * tyg
        + rotate_matrix[2, 2] * tzg
    )

    #   vector in geo coordinate
    v = (
        -np.sin(lat) * np.cos(lon) * txr
        - np.sin(lat) * np.sin(lon) * tyr
        + np.cos(lat) * tzr
    )
    u = -np.sin(lon) * txr + np.cos(lon) * tyr

    u = np.array(u)
    v = np.array(v)

    return (u, v)

In [35]:
def scalar_g2r(al, be, ga, lon, lat):
    """
    Converts geographical coordinates to rotated coordinates.
    Parameters
    ----------
    al : float
        alpha Euler angle
    be : float
        beta Euler angle
    ga : float
        gamma Euler angle
    lon : array
        1d array of longitudes in geographical coordinates
    lat : array
        1d array of latitudes in geographical coordinates
    Returns
    -------
    rlon : array
        1d array of longitudes in rotated coordinates
    rlat : array
        1d araay of latitudes in rotated coordinates
    """

    rad = mt.pi / 180
    al = al * rad
    be = be * rad
    ga = ga * rad

    rotate_matrix = np.zeros(shape=(3, 3))

    rotate_matrix[0, 0] = np.cos(ga) * np.cos(al) - np.sin(ga) * np.cos(be) * np.sin(al)
    rotate_matrix[0, 1] = np.cos(ga) * np.sin(al) + np.sin(ga) * np.cos(be) * np.cos(al)
    rotate_matrix[0, 2] = np.sin(ga) * np.sin(be)
    rotate_matrix[1, 0] = -np.sin(ga) * np.cos(al) - np.cos(ga) * np.cos(be) * np.sin(
        al
    )
    rotate_matrix[1, 1] = -np.sin(ga) * np.sin(al) + np.cos(ga) * np.cos(be) * np.cos(
        al
    )
    rotate_matrix[1, 2] = np.cos(ga) * np.sin(be)
    rotate_matrix[2, 0] = np.sin(be) * np.sin(al)
    rotate_matrix[2, 1] = -np.sin(be) * np.cos(al)
    rotate_matrix[2, 2] = np.cos(be)

    lat = lat * rad
    lon = lon * rad

    # geographical Cartesian coordinates:
    xr = np.cos(lat) * np.cos(lon)
    yr = np.cos(lat) * np.sin(lon)
    zr = np.sin(lat)

    # rotated Cartesian coordinates:
    xg = rotate_matrix[0, 0] * xr + rotate_matrix[0, 1] * yr + rotate_matrix[0, 2] * zr
    yg = rotate_matrix[1, 0] * xr + rotate_matrix[1, 1] * yr + rotate_matrix[1, 2] * zr
    zg = rotate_matrix[2, 0] * xr + rotate_matrix[2, 1] * yr + rotate_matrix[2, 2] * zr

    # rotated coordinates:
    rlat = np.arcsin(zg)
    rlon = np.arctan2(yg, xg)

    a = np.where((np.abs(xg) + np.abs(yg)) == 0)
    if a:
        lon[a] = 0

    rlat = rlat / rad
    rlon = rlon / rad

    return (rlon, rlat)

In [93]:
mesh_path = '/work/ab0995/a270046/fesom2-meshes/NEMO_ecmwf/NEMO/'
ufile = '/work/bk1040/DYAMOND/.input_winter_data/IFS-FESOM2-4km/u.fesom.2020.nc'
vfile = '/work/bk1040/DYAMOND/.input_winter_data/IFS-FESOM2-4km/v.fesom.2020.nc'
ofolder = '/mnt/lustre01/work/ab0995/a270088/DYAMOND/ROTATE_UV/IFS-FESOM2-4km/'

In [6]:
nodes = pd.read_csv(os.path.join(mesh_path,'nod2d.out'), skiprows=1, delim_whitespace=True, names=['n','x','y','f'])

In [10]:
elem = np.loadtxt(os.path.join(mesh_path,'elem2d.out'), skiprows=1)

In [12]:
elem = (elem-1).astype('int')

In [57]:
fmesh = namedtuple("mesh", "x2 y2 elem")
mesh = fmesh(x2=nodes.x.values.astype('float32'), y2=nodes.y.values.astype('float32'), elem=elem)

In [58]:
face_x, face_y = compute_face_coords(mesh)

In [60]:
u = xr.open_dataset(ufile)
v = xr.open_dataset(vfile)

In [61]:
ut = u.u[0,:,0]
vt = v.v[0,:,0]

In [62]:
urot, vrot = vec_rotate_r2g(50, 15, -90, face_x, face_y, ut, vt, flag=1)

In [84]:
time = 1
out_u = xr.Dataset({'u':(['time','elem','nz1'],u3d.astype('float32'), u.u.attrs)},
          coords={'time':[u.time[time].data],
                 }, attrs=u.attrs)

In [104]:
def fesoro(time):
    
    u3d = np.zeros((1,u.u.shape[1], u.u.shape[2])).astype('float32')
    v3d = np.zeros((1,u.u.shape[1], u.u.shape[2])).astype('float32')
    
    u0 = u.u[time,:,:].data
    v0 = v.v[time,:,:].data
    
    for i in range(47):
        ut = u0[:,i]
        vt = v0[:,i]
        urot, vrot = vec_rotate_r2g(50, 15, -90, face_x, face_y, ut, vt, flag=1)
        u3d[0,:,i] = urot
        v3d[0,:,i] = vrot
        print(i)
    out_u = xr.Dataset({'u':(['time','elem','nz1'], u3d.astype('float32'), u.u.attrs)},
          coords={'time':[u.time[time].data],
                 }, attrs=u.attrs)
    out_v = xr.Dataset({'v':(['time','elem','nz1'], v3d.astype('float32'), v.v.attrs)},
          coords={'time':[v.time[time].data],
                 }, attrs=v.attrs)
    
    out_u.to_netcdf(f'{ofolder}/u_{str(time).zfill(10)}.nc', encoding={'u': {'dtype': 'float32', '_FillValue': None}})
    out_v.to_netcdf(f'{ofolder}/v_{str(time).zfill(10)}.nc', encoding={'v': {'dtype': 'float32', '_FillValue': None}})

In [105]:
from joblib import Parallel, delayed

In [111]:
u.u.shape

(320, 1801435, 47)

In [113]:
%%time
Parallel(n_jobs=20)(delayed(fesoro)(i) for i in range(u.u.shape[0]))

CPU times: user 3.37 s, sys: 785 ms, total: 4.15 s
Wall time: 16min 20s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [114]:
uu = xr.open_mfdataset(f'{ofolder}/u_*.nc', chunks={'time':30})

In [115]:
uu

<xarray.Dataset>
Dimensions:  (elem: 1801435, nz1: 47, time: 320)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20T02:40:00 ... 2020-02-28T23:40:00
Dimensions without coordinates: elem, nz1
Data variables:
    u        (time, elem, nz1) float32 dask.array<chunksize=(1, 1801435, 47), meta=np.ndarray>
Attributes:
    history:  Mon Oct 11 14:46:48 2021: ncatted -a axis,u,c,c,T-Z u.fesom.202...
    NCO:      netCDF Operators version 4.9.3 (Homepage = http://nco.sf.net, C...

In [116]:
uu.to_netcdf(f'{ofolder}/out/uout.nc')

In [117]:
vv = xr.open_mfdataset(f'{ofolder}/v_*.nc', chunks={'time':30})

In [118]:
vv.to_netcdf(f'{ofolder}/out/vout.nc')